### Install these libraries 

In [ ]:
# !pip -q install langchain openai tiktoken PyPDF2 faiss-cpu

# Importing necessary Libraries

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import os
import openai
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS 

In [ ]:
OPENAI_API_KEY= "Put your openapi key"

In [ ]:
folder = r'allpdfs'
os.listdir(folder)

In [ ]:
loaders = [UnstructuredPDFLoader(os.path.join(folder, fn)) for fn in os.listdir(folder)]
loaders

In [ ]:
texts = []
for i in range(len(loaders)):
    data = loaders[i].load()
    texts.append(data[0])
print(len(texts))

# Splitting the texts

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunks = text_splitter.split_documents(texts)

In [ ]:
len(chunks)

In [ ]:
chunks[20]

In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# doc_search = Chroma.from_documents(chunks,embeddings)
docsearch = FAISS.from_texts([text.page_content for text in chunks], embeddings)

In [ ]:
docsearch.embedding_function

In [ ]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
chain = load_qa_chain(llm,chain_type="stuff")

# Output

In [ ]:
while True:
    query =  input("Enter the questions: ")
    docs = docsearch.similarity_search(query)
    answer = chain.run(input_documents =docs, question= query )
    print(answer)
    print()